In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import folium
import branca.colormap as cm
import geopandas as gpd
import pandas as pd
from folium.features import CustomIcon
from folium.plugins import FloatImage

import time
import csv
import pathlib
import os
import re


In [2]:
# Estaciones según numeración del ayuntamiento
estaciones = [4, 8, 11, 16, 17, 18, 24, 27, 35, 36, 38, 39, 40, 47, 48,
              49, 50, 54, 55, 56, 57, 58, 59, 60]

# Nombres correspondientes a cada una de las estaciones
estaciones_nombres = ['Plaza España', 'Escuelas Aguirre',
                      'Av. Ramón y Cajal', 'Arturo Soria',
                      'Villaverde Alto', 'Calle Farolillo',
                      'Casa de Campo', 'Barajas', 'Pza. del Carmen',
                      'Moratalaz', 'Cuatro Caminos',
                      'Barrio del Pilar', 'Vallecas', 'Méndez Álvaro',
                      'Pº. Castellana', 'Retiro', 'Pza. Castilla',
                      'Ensanche Vallecas', 'Urb. Embajada (Barajas)',
                      'Plaza Elíptica', 'Sanchinarro', 'El Pardo',
                      'Parque Juan Carlos I', 'Tres Olivos']

# Diccionario de las estaciones con su código de distrito correspondiente
dict_estaciones = {'Plaza España': '09', 'Escuelas Aguirre': '04',
                   'Av. Ramón y Cajal': '05',
                   'Arturo Soria': '15', 'Villaverde Alto': '17',
                   'Calle Farolillo': '11', 'Casa de Campo': '09',
                   'Barajas': '21', 'Pza. del Carmen': '01',
                   'Moratalaz': '14', 'Cuatro Caminos': '07',
                   'Barrio del Pilar': '08', 'Vallecas': '13',
                   'Méndez Álvaro': '02', 'Pº. Castellana': '05',
                   'Retiro': '03', 'Pza. Castilla': '06',
                   'Ensanche Vallecas': '18',
                   'Urb. Embajada (Barajas)': '21',
                   'Plaza Elíptica': '12', 'Sanchinarro': '16',
                   'El Pardo': '08', 'Parque Juan Carlos I': '21',
                   'Tres Olivos': '08'}

# Coordenadas geográficas de las estaciones
coordenadas = {
    'Estacion': ['Plaza España', 'Escuelas Aguirre', 'Av. Ramón y Cajal',
                 'Arturo Soria', 'Villaverde Alto', 'Calle Farolillo',
                 'Casa de Campo', 'Barajas', 'Pza. del Carmen',
                 'Moratalaz',
                 'Cuatro Caminos', 'Barrio del Pilar', 'Vallecas',
                 'Méndez Álvaro', 'Pº. Castellana', 'Retiro',
                 'Pza. Castilla',
                 'Ensanche Vallecas', 'Urb. Embajada (Barajas)',
                 'Plaza Elíptica', 'Sanchinarro', 'El Pardo',
                 'Parque Juan Carlos I', 'Tres Olivos'],
    'Longitud': ['3°42\'43.91"O', '3°40\'56.22"O', '3°40\'38.5"O',
                 '3°38\'21.17"O', '3°42\'47.89"O', '3°43\'54.61"O',
                 '3°44\'50.44"O', '3°34\'48.10"O', '3°42\'11.4"O',
                 '3°38\'43.02"O', '3°42\'25.64"O', '3°41\'41.53"O',
                 '3°39\'5.5"O', '3°41\'12.57"O', '3°41\'25.34"O',
                 '3°40\'57.3"O', '3°41\'19.48"O', '3°36\'43.7"O',
                 '3°34\'50.03"O', '3°43\'7.54"O', '3°39\'37.86"O',
                 '3°46\'28.62"O', '3°36\'32.66"O', '3°41\'23.03"O'],
    'Latitud': ['40°25\'25.98"N', '40°25\'17.63"N', '40°27\'5.29"N',
                '40°26\'24.20"N', '40°20\'49.74"N', '40°23\'41.22"N',
                '40°25\'9.69"N', '40°28\'36.93"N', '40°25\'9.15"N',
                '40°24\'28.64"N', '40°26\'43.97"N', '40°28\'41.64"N',
                '40°23\'17.33"N', '40°23\'53.17"N', '40°26\'23.61"N',
                '40°24\'52"N', '40°27\'56.1"N', '40°22\'22.48"N',
                '40°27\'44.51"N', '40°23\'6.1"N', '40°29\'39.12"N',
                '40°31\'4.97"N', '40°27\'54.90"N', '40°30\'1.97"N']}

# Diccionario con los gases y sus claves en las tablas del Ayuntamiento
claves_gases = {1:'SO2', 6:'CO', 7:'NO', 8:'NO2', 9:'PM2.5', 10:'PM10', 12:'NOx', 14:'O3',
                20:'TOL', 30:'BEN', 35:'EBE', 37:'MXY', 38:'PXY', 39:'OXY', 42:'TCH', 43:'CH4'}

meses = ['ene', 'feb','mar','abr','may','jun','jul','ago','sep','oct','nov','dic']


In [3]:
def LatLon2UTM(s):
    '''
    Convertimos las coordenadas en formato UTM a º, ' y ".
    '''
    
    #Metodo para transformar coordenadas en º, ' y " a coordenadas UTM
    # example: s = """0°51'56.29"S"""
    degrees, minutes, seconds, direction = re.split('[°\'"]+', s)
    dd = float(degrees) + float(minutes) / 60 + float(seconds) / (60 * 60);
    if direction in ('S', 'O'):
        dd *= -1
    return dd

In [4]:
def drawMap(data_maximos_dia, mes, year, gas, coordenadas_estacion, dia_mes):
    
    '''
    Dibujamos un mapa en html donde en una escala de colores podemos observar fácilmente los valores máximos de 
    contaminación en cada unos de los distritos.
    '''
    
    if gas == 'NOx':
        lim = 400
        uni = '(µg/m3)'
    elif gas == 'SO2':
        lim = 500
        uni = '(µg/m3)'
    elif gas == 'CO':
        lim = 10
        uni = '(mg/m3)'
    elif gas == 'O3':
        lim = 240
        uni = '(µg/m3)'
    else:
        lim = None
        uni = '(µg/m3)'
    
    localitation = [40.4662822, -3.6896636]
    m = folium.Map(location=localitation, zoom_start=11)
    maximo_color = data_maximos_dia['Valor maximo'].max() + 0.000001
    geo_data = gpd.read_file('./Previous_files/Distritos.json')
    maximos_geo = geo_data.merge(data_maximos_dia, on="CODDISTRIT")
    # Ploteo de los valores enun mapa de color
    # Markers para las estaciones
    nombre_gas = ''
    dict_gases = claves_gases
    for gas_find, key in dict_gases.items():
        if key == gas:
            nombre_gas = gas_find

    for ind in data_maximos_dia.index:
        icon_image = './Previous_files/map-pin-solid.png'
        icon = CustomIcon(icon_image, icon_size=(15, 30))
        folium.Marker(location=[coordenadas_estacion['Latitud'][ind],
                                coordenadas_estacion['Longitud'][ind]],
                     icon=icon,
                     popup=str(data_maximos_dia['Estacion'][ind])
                     ).add_to(m)

    colors = [(169, 208, 142), (255, 217, 102), (255, 71, 74), (157, 0, 0)]
    index = [0, 101, 201, 301, maximo_color]
    data_color = data_maximos_dia['Valor maximo']
    colormap = cm.StepColormap(colors, index, vmin=0, vmax=maximo_color,
                               caption=('Niveles de %s %s' % (gas, uni)))
    colormap.add_to(m)
    style_function = lambda x: {"weight": 0.5, 'color': 'black',
                               'fillColor': colormap(x['properties']['Valor maximo']),
                               'fillOpacity': 0.5}

    highlight_function = lambda x: {'fillColor': '#000000',
                                   'color': '#000000',
                                   'fillOpacity': 0.50, 'weight': 0.1}
    NIL = folium.features.GeoJson(maximos_geo,
                                 style_function=style_function,
                                 control=False,
                                 tooltip=folium.features.GeoJsonTooltip(
                                     fields=['CODDISTRIT', 'Valor maximo'], 
                                     aliases=['Codigo Distrito: ', 'Valor maximo%s: '%uni],
                                     style=('background-color: white; '
                                            'color: #333333; '
                                            'font-family: '
                                            'arial; font-size: 12px; '
                                            'padding: 10px;'), 
                                            sticky=True
                                 )
                                 )
    m.add_child(NIL)
    m.keep_in_front(NIL)
    folium.LayerControl().add_to(m)
    if nombre_gas == 'NOx':
        image_file = './Previous_files/Niveles.png'
        FloatImage(image_file, bottom=73, left=69.8).add_to(m)

    if os.path.isdir('./Resultados/contaminacion_madrid_%s_gas_%s/' % (mes, gas)):
        pass
    else:
        os.mkdir('./Resultados/contaminacion_madrid_%s_gas_%s/' % (mes, gas))

    m.save('Resultados/contaminacion_madrid_%s_gas_%s/dia_%d.html' % (mes, gas, dia_mes+1))


In [5]:
def maximos_distritos(data_maximos_dia, mes, year, gas, coordenadas_estacion, index):
    
    '''
    Obtenemos el valor medio en los distritos que tienen más de una estación de medición.
    '''
    

    lista_distritos = ['05', '08', '09', '21']
    for i in lista_distritos:
        valor_maximo_distrito = []
        indice_rep = []
        for j in range(0, len(data_maximos_dia)):
            distrito_b = data_maximos_dia['CODDISTRIT'][j]
            if distrito_b == i:
                indice_rep.append(j)
                valor_maximo_distrito.append(data_maximos_dia['Valor maximo'][j])
            else:
                   continue
            valor_maximo_distrito_i = sum(valor_maximo_distrito) / len(valor_maximo_distrito)
            for k in indice_rep:
                data_maximos_dia['Valor maximo'][k] = valor_maximo_distrito_i
           
            for ind in data_maximos_dia.index:
                if np.isnan(data_maximos_dia['Valor maximo'].iloc[ind]):
                    data_maximos_dia['Valor maximo'].iloc[ind] = 0

        
    drawMap(data_maximos_dia, mes, year, gas, coordenadas_estacion, index)

In [6]:
year = input('Introduce el año a estudiar: \n19\n20\n\n\n') 

print('\n\nTenga en cuenta que no todas las estaciones de medición tienen la capacidad de medir todos los gases.\n'
        'Para más información, consultar: https://datos.madrid.es/portal/site/egob/ \n\n')


gas_num = int(input('A continuación escribe el código asociado al gas que quieres representar: \n\n'
            'SO2 -> 1  ;   CO -> 6  ;   NO -> 7\n' 
            'NO2 -> 8  ;   PM2.5 -> 9 ; PM10 -> 10\n'
            'NOx -> 12  ;  O3 -> 14  ;  TOL -> 20\n'
            'BEN -> 30  ;  EBE -> 35  ; MXY -> 37\n'
            'PXY -> 38  ;  OXY -> 39  ; TCH -> 42\n'
            'CH4 -> 43\n'))

gas=claves_gases[gas_num]
for mes in meses:
    path = ('./Resultados/Graficas_%s_%s_gas_%s/' % (mes, year, gas))
    data_maximos_mes = pd.read_csv(path + ('tabla_maximos.csv'), sep=';')

    columnas = ['Estacion', 'CODDISTRIT', 'Valor maximo']
    data_maximos_mes.drop(['Unnamed: 0'], axis = 1, inplace=True)


    coordenadas_estacion = pd.DataFrame(coordenadas)
    coordenadas_estacion['Longitud'] = coordenadas_estacion['Longitud'].apply(LatLon2UTM)
    coordenadas_estacion['Latitud'] = coordenadas_estacion['Latitud'].apply(LatLon2UTM)


    for index in data_maximos_mes.index:
        data_maximos_dia = pd.DataFrame(columns = columnas, index = np.arange(0,len(estaciones),1))
        data_maximos_dia['Estacion'] = data_maximos_mes.columns 

        i = 0
        for est in data_maximos_dia['Estacion']:
            data_maximos_dia['CODDISTRIT'].iloc[i] = dict_estaciones[est]  
            data_maximos_dia['Valor maximo'].iloc[i] = data_maximos_mes[est].iloc[index]
            i += 1 


        maximos_distritos(data_maximos_dia, mes, year, gas, coordenadas_estacion, index)


Introduce el año a estudiar: 
19
20


19


Tenga en cuenta que no todas las estaciones de medición tienen la capacidad de medir todos los gases.
Para más información, consultar: https://datos.madrid.es/portal/site/egob/ 


A continuación escribe el código asociado al gas que quieres representar: 

SO2 -> 1  ;   CO -> 6  ;   NO -> 7
NO2 -> 8  ;   PM2.5 -> 9 ; PM10 -> 10
NOx -> 12  ;  O3 -> 14  ;  TOL -> 20
BEN -> 30  ;  EBE -> 35  ; MXY -> 37
PXY -> 38  ;  OXY -> 39  ; TCH -> 42
CH4 -> 43
12
